In [32]:
import cv2
from PIL import Image

In [33]:
metodos = [
cv2.THRESH_BINARY,
cv2.THRESH_BINARY_INV,
cv2.THRESH_TRUNC,
cv2.THRESH_TOZERO,
cv2.THRESH_TOZERO_INV
]

In [35]:
img = cv2.imread("indice.png")

In [36]:

imagem_cinza = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

In [37]:
i=0
for metodo in metodos:
    i+=1
    _ , imagem_tratada = cv2.threshold(imagem_cinza,127,255, metodo or cv2.THRESH_OTSU)
    
    cv2.imwrite(f"imagem_tratada_{i}.png",imagem_tratada)

In [27]:
imagem = Image.open("imagem_tratada_3.png")

In [28]:
imagem = imagem.convert("P")

In [29]:
imagem2 = Image.new("P",imagem.size,255)

In [31]:
for x in range(imagem.size[1]):
    for y in range(imagem.size[0]):
        cor_pixel = imagem.getpixel((y,x))
        if cor_pixel < 115:
            imagem2.putpixel((y,x),0)
imagem2.save('imagemfinal.png')